# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect("wine.db")
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
customer:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
place:
  0: LIEU(TEXT)
  1: REGION(TEXT)
production:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
quantity:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NB(NUM)
  1: DATES(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: NV(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [4]:
%reload_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM MASTER1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [79]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

[Write your answer here]

## Pre-processing
Before handling wine's database, we have to :
- convert table master 1 and 2 to the 2nd and 3rd normal form (as possible)
- explain functionnal dependencies <br/>
<p> <b>Why this preprocessing ?</b> </p>
- avoid repetitive datas 
- avoid writing and reading mistakes
- optimise system (queries...)

The schedule below shows all the dependencies between the tables created.
We distinguish two kind of tables depending their origin : master 1 or master 2.
<p><b>Note</b>: i've introduce the notion of "composed key" to clarly identify primary key with unique ID.</p>

![](Schedule_wine_database.jpg)

![](Schedule_wine_database.png)

In [115]:
%%sql
DROP TABLE IF EXISTS wine;
CREATE TABLE wine AS
SELECT DISTINCT NV, CRU, DEGRE, MILL 
FROM Master1
WHERE NV IS NOT NULL;

SELECT * 
FROM wine
ORDER BY NV ASC
LIMIT 5;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976


In [116]:
%%sql 
DROP TABLE IF EXISTS production;
CREATE TABLE production AS
SELECT DISTINCT NP, NOM, PRENOM, REGION 
FROM Master1
WHERE NP IS NOT NULL;

SELECT * 
FROM production
ORDER BY NP ASC
LIMIT 5;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NOM,PRENOM,REGION
1,Bohn,Rene,Alsace
2,Boxler,Albert,Alsace
3,Six,Paul,Alsace
4,Stentz,Fernand,Alsace
5,Joudeat,Lucien,Bourgogne


In [117]:
%%sql 
DROP TABLE IF EXISTS quantity;
CREATE TABLE quantity AS
SELECT DISTINCT NP, NV, QTE 
FROM Master1
WHERE NP IS NOT NULL AND NV IS NOT NULL;

SELECT * 
FROM quantity
ORDER BY QTE DESC
LIMIT 5;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NV,QTE
107,107,7000
7,19,986
24,98,986
85,43,983
85,64,980


#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

[Write your answer here]

In [118]:

%%sql
SELECT *
FROM master2
LIMIT 5;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,DATES,LIEU,QTE,NB,NOM,PRENOM,TYPE,REGION
None,None,None,None,None,None,None,11,Breton,Andre,petit,None
None,None,None,None,None,None,None,13,Barthes,Roland,moyen,None
None,None,None,None,None,None,None,16,Balzac,Honore de,moyen,None
None,None,None,None,None,None,None,18,Celine,Louis Ferdinand,gros,None
None,None,None,None,None,None,None,20,Chateaubriand,Francois-Rene de,moyen,None


In [14]:
%%sql
DROP TABLE IF EXISTS customer;
CREATE TABLE customer AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE 
FROM Master2
WHERE NB IS NOT NULL;

SELECT * 
FROM customer
LIMIT 5;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen


In [15]:
%%sql 
DROP TABLE IF EXISTS place;
CREATE TABLE place AS
SELECT DISTINCT LIEU, REGION 
FROM Master2
WHERE LIEU IS NOT NULL AND REGION IS NOT NULL;
SELECT *
FROM place
LIMIT 5;

 * sqlite:///wine.db
Done.
Done.
Done.


LIEU,REGION
BORDEAUX,NOUVELLE-AQUITAINE
PARIS,ÎLE-DE-FRANCE
RENNES,BRETAGNE
LYON,AUVERGNE-RHÔNE-ALPES
NICE,PROVENCE-ALPES-CÔTE D'AZUR


In [16]:
%%sql 
DROP TABLE IF EXISTS sales;
CREATE TABLE sales AS
SELECT DISTINCT NB, DATES, LIEU, QTE, NV 
FROM Master2
WHERE LIEU IS NOT NULL AND NB IS NOT NULL;
SELECT *
FROM sales
LIMIT 5;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,DATES,LIEU,QTE,NV
2,1977-11-02,BORDEAUX,33,1
44,2015-10-16,PARIS,1,1
45,1983-12-31,RENNES,1,1
48,1983-12-25,LYON,2,2
7,1978-11-01,NICE,6,3


Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [119]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [18]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
customer:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
place:
  0: LIEU(TEXT)
  1: REGION(TEXT)
production:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
quantity:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NB(NUM)
  1: DATES(NUM)
  2: LIEU(TEXT)
  3: QTE(NUM)
  4: NV(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


In [121]:
%%sql
SELECT *
FROM dummy
LIMIT 5;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [122]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy
LIMIT 5;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7


Notice that we use `NULL` given that `None` is only used for display.

In [123]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

We created it above.

#### Exercise 1.4

Create the new tables from Master2:

We created it above.

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [22]:
%%sql SELECT DISTINCT type
FROM customer;

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [125]:
%%sql
SELECT CRU, REGION
FROM quantity JOIN production ON quantity.NP = production.NP
              JOIN wine ON quantity.NV = wine.NV
WHERE CRU="Pommard" or CRU ="Brouilly"

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [149]:
%%sql
SELECT DISTINCT REGION
FROM quantity JOIN production ON quantity.NP = production.NP
              JOIN wine ON quantity.NV = wine.NV 
GROUP BY CRU
HAVING CRU in ("Pommard","Brouilly")
    

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [151]:
%%sql
SELECT wine.cru, wine.MILL, SUM(sales.QTE) 
FROM sales JOIN Wine ON wine.NV = sales.NV
GROUP BY sales.NV
ORDER BY wine.MILL DESC;


 * sqlite:///wine.db
Done.


CRU,MILL,SUM(sales.QTE)
Chenas,1984,1
Beaujolais Primeur,1983,7
Mercurey,1981,36
Cotes de Provence,1980,18
Tavel,1980,17
Tavel,1980,16
Rasteau,1980,15
Saint Amour,1980,14
Cornas,1980,13
Cornas,1980,12


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [152]:
%%sql 
SELECT quantity.NV, count(production.NP)
FROM quantity JOIN production ON quantity.NP = production.NP
GROUP BY quantity.NV
HAVING count(production.NP)>3;


 * sqlite:///wine.db
Done.


NV,count(production.NP)
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [153]:
%%sql
SELECT production.NP, production.NOM, production.PRENOM
FROM production LEFT JOIN quantity ON quantity.NP = production.NP
WHERE quantity.QTE is null
ORDER BY production.NP ASC;

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
4,Stentz,Fernand
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
12,Tortochot,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [159]:
%%sql
SELECT DISTINCT customer.NB, wine.MILL, customer.NOM, customer.PRENOM
FROM sales JOIN customer ON customer.NB = sales.NB
            JOIN wine ON wine.NV = sales.NV
WHERE wine.MILL=1980
ORDER BY customer.NB ASC;

 * sqlite:///wine.db
Done.


NB,MILL,NOM,PRENOM
2,1980,Artaud,Antonin
8,1980,Aragon,Louis
44,1980,Gide,Andre
45,1980,Giono,Jean
50,1980,Lautreamont,None
61,1980,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [164]:
%%sql
SELECT DISTINCT customer.NB, customer.NOM, customer.PRENOM
FROM sales inner JOIN customer ON customer.NB = sales.NB
       inner JOIN wine ON wine.NV = sales.NV
WHERE wine.MILL!=1980
ORDER BY customer.NB ASC;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
2,Artaud,Antonin
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
8,Aragon,Louis
9,Ajar,Emile
10,Andersen,Yann


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [185]:
%%sql
SELECT DISTINCT customer.NB, customer.NOM, customer.PRENOM, wine.MILL
FROM sales INNER JOIN Customer ON customer.NB = sales.NB 
INNER JOIN wine on wine.NV = sales.NV
WHERE mill=1980;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,MILL
2,Artaud,Antonin,1980
44,Gide,Andre,1980
45,Giono,Jean,1980
8,Aragon,Louis,1980
50,Lautreamont,None,1980
61,Mallarme,Stephane,1980


#### Exercise 2.10

List all wines from 1980

In [41]:
%%sql
SELECT NV, CRU, MILL, DEGRE
FROM wine
WHERE MILL=1980

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [165]:
%%sql
SELECT sales.NV, wine.CRU, wine.DEGRE, wine.MILL
FROM sales JOIN customer ON customer.NB = sales.NB
            JOIN wine ON sales.NV = wine.NV
WHERE customer.NB = 2 and wine.mill=1980;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [182]:
%%sql
SELECT customer.NB, customer.NOM, customer.PRENOM, count(wine.NV)
FROM sales JOIN customer ON customer.NB = sales.NB JOIN wine ON sales.NV = wine.NV
WHERE wine.mill = 1980
GROUP BY customer.NB
HAVING count(wine.NV) = (select count(NV) FROM wine WHERE mill=1980);

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,count(wine.NV)
44,Gide,Andre,18
